<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Apprentissage non paramétrique en régression et régularisation

** Objectifs : ** 

Comparaison sur le même jeu de données des qualités de prévision de plusieurs modèles obtenus par:

- **Des méthodes unidimensionnelles :**

    - Splines
    - Estimateurs à noyaux
    - Estimation ponctuelle par polynômes locaux
    
    
- **Des méthodes multidimensionnelles :**

   - Modèle linéaire
   - Modèles additifs généralisés
   - Arbre de régression CART
   - Kernel Regression Least Square
   - Estimateurs à noyaux multidimensionnels.



# Les données

Nous utiliserons les données ** airquality ** du logiciel R. Ces données correspondent à des mesures quotidiennes de la qualité de l'air.
La variable à expliquer est le taux d'ozone, les variables explicatives sont 
- Le rayonnement solaire
- La température
- La vitesse du vent 



** Liste des 6 variables, la première est à expliquer **

- Ozone :  Mean ozone in parts per billion from 1300 to 1500 hours at Roosevelt Island 
- Solar.R :   Solar radiation in Langleys in the frequency band 4000 to 7700  Angstroms from 08.00 to 12.00 hours at  Central Park 
- Wind :  Average wind speed in miles per hour at 07.00 and 10.00 hours atLa Guardia Airport
- Temp : Maximum daily temperature in degrees Fahrenheit at La Guardia Airport
- Month : Numeric Month (1 - 12)\\
- Day : Numeric Day of month (1 - 31)





In [ ]:
data(airquality)
summary(airquality)
help(airquality)
airquality

## Suppression  des données manquantes et atypiques 

In [ ]:
airquality=airquality[complete.cases(airquality),]
airquality

airquality=airquality[airquality$Ozone<110,]

## Description des données

In [ ]:
dim(airquality)
cor(airquality)

##  Création d'un échantillon test

In [ ]:
ind_test=4*c(1:25)
airquality_app=airquality[-ind_test,]
airquality_test=airquality[ind_test,]

# Modélisation unidimensionnelle

Dans cette partie, nous allons mettre en oeuvre des méthodes nonparamétriques pour modéliser 
la variable Ozone en fonction de la Température. 

In [ ]:
plot(airquality_app$Temp,airquality_app$Ozone)

## Splines

In [ ]:
Ozone=airquality_app$Ozone
Temp=airquality_app$Temp
plot(Temp,Ozone, main="Smoothing Splines")
Ozone.spl <- smooth.spline(Temp, Ozone)
Ozone.spl
lines(Ozone.spl, col = "blue")
lines(smooth.spline(Temp, Ozone, df=15), lty=2,
 col = "red")
legend(60,100,c(paste("default [C.V.] => df =",
round(Ozone.spl$df,1))," df = 15"),
 col = c("blue","red"), lty = 1:2)

** Q. ** Essayer d'autres valeurs de df  : 2 puis 20.

### Calcul de l'erreur d'apprentissage

In [ ]:
pred.app.spl=predict(Ozone.spl,Temp) 
sqrt(mean((pred.app.spl$y-Ozone)**2))

### Calcul de l'erreur sur l'échantillon  test : 

In [ ]:
Temp_test=airquality_test$Temp
Ozone_test=airquality_test$Ozone
pred.spl=predict(Ozone.spl,Temp_test) 
sqrt(mean((pred.spl$y-Ozone_test)**2))


## Estimateurs à noyaux

In [ ]:
Ozone.ker <- ksmooth(Temp, Ozone,kernel="normal",
bandwidth=5)
Ozone.ker
Ozone.ker2 <- ksmooth(Temp, Ozone,kernel="normal",
bandwidth=10)
plot(Temp,Ozone, main="Noyau gaussien")
lines(Ozone.ker,col="blue")
lines(Ozone.ker2,col="red",lty=2)

** Q. ** Faire la même chose avec le noyau ``box''.

In [ ]:
 # Pas de fonction ``predict''! On programme la fonction qui définit l'estimateur.

noyau=function(z,X,Y,h)
#z est la valeur en laquelle on calcule
#l'estimateur, h la fenêtre, 
#(X,Y) l'échantillon d'apprentissage. 
{
Vect=exp(-((X-z)**2)/(2*h**2))
noyau=(sum(Y*Vect))/(sum(Vect))
}

### Calcul de l'erreur d'apprentissage

In [ ]:
m=length(Temp)
pred.ker.app=c(rep(0,m))
for (j in 1:m){
pred.ker.app[j]=noyau(Temp[j],Temp,Ozone,5)}

sqrt(mean((pred.ker.app-Ozone)**2))


### Calcul de l'erreur sur l'échantillon  test : 

In [ ]:
p=length(Temp_test)
pred.ker.test=c(rep(0,p))
for (j in 1:p){
pred.ker.test[j]=noyau(Temp_test[j],Temp,Ozone,5)}

sqrt(mean((pred.ker.test-Ozone_test)**2))

## Polynômes locaux

Estimation ponctuelle par des polynômes locaux. 

In [ ]:
plot(Temp,Ozone, main="Polynomes locaux")
polyloc=loess(Ozone~Temp,span=0.5,family="gaussian")
pred_app=
predict(polyloc,data.frame(Temp=sort(unique(Temp))))
lines(sort(unique(Temp)),pred_app,col="blue")


** Q. ** Modifier le paramètre span et le noyau. 

In [ ]:
pred=predict(polyloc,data.frame(Temp))
plot(Ozone,pred)
abline(0,1)

### Calcul de l'erreur d'apprentissage

In [ ]:
pred.app.loess=predict(polyloc,Temp) 
sqrt(mean((pred.app.loess-Ozone)**2))

### Calcul de l'erreur sur l'échantillon  test : 

In [ ]:
pred.loess=predict(polyloc,Temp_test) 
sqrt(mean((pred.loess-Ozone_test)**2))

In [ ]:
# Attention aux éventuelles données manquantes. Si nécessaire :

ind=complete.cases(pred.loess)
sqrt(mean((pred.loess[ind]-Ozone_test[ind])**2))


# Modélisation multidimensionnelle

Nous allons ici prendre en compte l'ensemble des variables explicatives : température, rayonnement solaire, vitesse du vent. 

## Modèle linéaire : 

- Ajuster un modèle linéaire pour modéliser la variable ** Ozone **
en fonction des variables   ** température, vitesse du vent et rayonnement solaire ** tout 
d'abord sans interaction  puis en introduisant les interactions entre les variables.

- Estimer l'erreur sur l'échantillon d'apprentissage et sur l'échantillon test.

## Modèles Additifs Généralisés


In [ ]:
library(mgcv)
b = gam(Ozone~s(Solar.R)+s(Wind)+s(Temp),
data=airquality_app)
summary(b)
plot(b)

** Q. ** En utilisant la fonction ** `predict ` **, calculer l'erreur d'apprentissage et l'erreur sur l'échantillon test. 

## Arbres CART

In [ ]:
library(rpart)
fit = rpart(Ozone ~ Temp + Solar.R + Wind, 
data=airquality_app)
summary(fit)
print(fit)
plot(fit)
text(fit, use.n=TRUE)

** Q. ** En utilisant la fonction ** `predict ` **, calculer l'erreur d'apprentissage et l'erreur sur l'échantillon test. 

In [ ]:
# On obtient un arbre plus ou moins complexe en réglant le paramètre control :

fit = rpart(Ozone ~ Temp + Solar.R + Wind, data=airquality_app,control=rpart.control(cp=0.001))


** Q. ** Calculer l'erreur d'apprentissage et l'erreur sur l'échantillon test avec cet arbre.

## Kernel Regression Least Square

Par défaut, la fonction ** `krls`** du package ** `KRLS` ** utilise le noyau Gaussien. 
On peut aussi utiliser un noyau linéaire ou polynomial. 


In [ ]:
Solar.R= airquality_app$Solar.R
Wind=airquality_app$Wind
Temp=airquality_app$Temp
X_app=cbind(Solar.R,Wind,Temp)
Ozone=airquality_app$Ozone

library(KRLS)
krlsfit = krls(X=X_app,y=Ozone)
pred.app.krls=predict(object=krlsfit,newdata=X_app)


### Calcul de l'erreur d'apprentissage

In [ ]:
err_app=sqrt(mean((pred.app.krls$fit-Ozone)**2))
err_app

### Calcul de l'erreur sur l'échantillon  test : 

In [ ]:
Solar.R_test= airquality_test$Solar.R
Wind_test=airquality_test$Wind
Temp_test=airquality_test$Temp
X_test=cbind(Solar.R_test,Wind_test,Temp_test)
Ozone_test=airquality_test$Ozone

pred.test.krls=predict(object=krlsfit,newdata=X_test)

err_test=sqrt(mean((pred.test.krls$fit-Ozone_test)**2))
err_test

** Q. ** Comparer ces résultats avec ceux que l'on obtient pour un noyau linéaire ou polynomial. 


## Estimateurs à noyaux

L'estimateur à noyau multidimensionnel en régression n'est pas programmé. 
Nous programmons donc cet estimateur.

In [ ]:
#Fonction intermédiaire qui calcule une norme pondérée :

norm<-function(y){
h1=15
h2=5
h3=10
n=(y[1]^2)/(2*h1**2)+(y[2]^2)/(2*h2**2)
+(y[3]^2)/(2*h3**2)
n}


In [ ]:
#Fonction qui calcule l'estimateur à noyau en un point z;
# X est la matrice des variables explicatives 
# y le vecteur des observations : 

noyau<-function(z,X,y){
m=length(y)
for (j in 1:m){
X[j,]=X[j,]-z}
nor=apply(X,1,norm) 
Vect2=-(nor)
f=sum(y*exp(-(nor)))
g=sum(exp(-(nor))) 
h=f/g
h}


In [ ]:
#Calcul de l'estimateur à noyau en les points 
#de l'échantillon d'apprentissage : 
m=length(Ozone)
est=rep(0,m)
for (j in 1:m){
z=rep(0,3)
z[1]=Solar.R[j]
z[2]=Wind[j]
z[3]=Temp[j]
z
est[j]=noyau(z,X_app,Ozone)
est[j]
}


In [ ]:
plot(Ozone,est)
abline(0,1,col=2)

### Calcul de l'erreur d'apprentissage :

In [ ]:
sqrt(mean((est-Ozone)**2))

### Calcul de l'erreur sur l'échantillon test :

In [ ]:
#Calcul de l'estimateur à noyau en les points 
#de l'échantillon test  : 
m_test=length(Ozone_test)
pred=rep(0,m_test)
for (j in 1:m_test){
z=rep(0,3)
z[1]=Solar.R_test[j]
z[2]=Wind_test[j]
z[3]=Temp_test[j]
pred[j]=noyau(z,X_app,Ozone)
pred[j]
}
plot(Ozone_test,pred)
lines(Ozone_test,Ozone_test,col=2)
sqrt(mean((pred-Ozone_test)**2))

- Les paramètres h1, h2, h3 choisis ne sont probablement pas optimaux, il faudrait optimiser ces paramètres. 
- Pb avec cette méthode : le fléau de la dimension. 
